In [68]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

import h5py
import sys
sys.path.append("/Users/diiego/software/WCSimDataTools/src")

from wcsimreader   import utils
from filters       import test_filters
from conversion    import descriptions
from wcsimwriter   import writer
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', 10000000)
pd.set_option('display.max_columns', 10000000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from IPython.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [4]:
filename  = "/Users/diiego/software/WCSimDataTools/ambeEvt_StackingOnly_1000evts_SFM0_2Th.h5"
nevents   = 1000
sfm       = 0
threshold = 2

In [6]:
utils.explore_file(filename)

/Users/diiego/software/WCSimDataTools/ambeEvt_StackingOnly_1000evts_SFM0_2Th.h5 (File) 'wcsim'
Last modif.: '2024-04-17T13:54:26+00:00'
Object Tree: 
/ (RootGroup) 'wcsim'
/Settings (Table(1,)) 'Settings'
/wcsimRootOptionsT (Table(1,)) 'wcsimRootOptionsT'
/wcsimGeoT (Group) 'wcsimGeoT'
/wcsimGeoT/Geometry (Table(1,)) 'Geometry'
/wcsimGeoT/PMT (Table(2014,)) 'PMT'
/wcsimT (Group) 'wcsimT'
/wcsimT/CherenkovHitTimes (Table(362937,)) 'CherenkovHitTimes'
/wcsimT/CherenkovHits (Table(320753,)) 'CherenkovHits'
/wcsimT/Tracks (Table(4481925,)) 'Tracks'
/wcsimT/Triggers (Table(10780,)) 'Triggers'
/wcsimT/CherenkovDigiHits (Group) 'CherenkovDigiHits'
/wcsimT/CherenkovDigiHits/DigiHits (Table(158470,)) 'DigiHits'
/wcsimT/CherenkovDigiHits/DigiHitsPhotonIdParent (VLArray(158470,)) 'PhotonIdParent'
/wcsimT/CherenkovDigiHits/PhotonIDs (VLArray(158470,)) 'PhotonIDs'



In [7]:
hittimes_df = utils.read_table(filename, "wcsimT/CherenkovHitTimes")
cherhits_df = utils.read_table(filename, "wcsimT/CherenkovHits")
digihits_df = utils.read_table(filename, "wcsimT/CherenkovDigiHits/DigiHits")
triggers_df = utils.read_table(filename, "wcsimT/Triggers")

tracks_df   = utils.read_table(filename, "wcsimT/Tracks")

settings_df = utils.read_table(filename, "Settings")
roptions_df = utils.read_table(filename, "wcsimRootOptionsT")
geometry_df = utils.read_table(filename, "wcsimGeoT/Geometry")
pmt_df      = utils.read_table(filename, "wcsimGeoT/PMT")

photonid_df = utils.read_vlarray(filename, "wcsimT/CherenkovDigiHits/PhotonIDs")
photparn_df = utils.read_vlarray(filename, "/wcsimT/CherenkovDigiHits/DigiHitsPhotonIdParent")

# Add PhotonIDParent to DigiHits DF and "beautify" tracks and hits DFs
photparn_list = [list(i) for i in photparn_df]
digihits_df['PhotonIDParent'] = photparn_list

tracks_df["CreatorProcessName"] = tracks_df["CreatorProcessName"].astype(str)
hittimes_df["PhotonCreatorProcessName"] = hittimes_df["PhotonCreatorProcessName"].astype(str)

In [8]:
digihits_df[digihits_df['EvtNum'].values == 0].head()

,Run,Date,EvtNum,SubEvtNumber,dhit,Q,T,TubeID,mPMTID,mPMT_PMTID,PhotonIDParent
0,0,5,0,1,0,1.146892,955.0,1443,76,18,[-1]
1,0,5,0,1,1,1.498578,955.8,1903,101,3,[165]
2,0,5,0,1,2,1.132836,957.0,1829,97,5,"[165, 165]"
3,0,5,0,1,3,3.706901,956.5,45,3,7,"[165, 165, 165]"
4,0,5,0,1,4,0.690179,954.5,1924,102,5,[165]


In [9]:
tracks_df[tracks_df['EvtNum'].values == 0][2:].sort_values('Id').head()

,Run,Date,EvtNum,SubEvtNumber,track,Ipnu,Flag,M,P,E,StartVol,StopVol,Dir_x0,Dir_x1,Dir_x2,PDir_x0,PDir_x1,PDir_x2,Stop_x0,Stop_x1,Stop_x2,Start_x0,Start_x1,Start_x2,Parenttype,ParentId,Time,Id,CreatorProcessName
73,0,5,0,1,73,2112,0,939.565369,87.472809,943.628418,10,10,-0.450578,0.010079,-0.892680,-39.413345,8.816043e-01,-7.808524e+01,-7.710765,-1.583446,-17.770853,0.000000,0.000000,0.000000,0,0,0.000000,1,
2,0,5,0,1,2,22,0,0.000000,4.400000,4.400000,10,10,0.806246,0.511164,0.297790,3.547481,2.249123e+00,1.310277e+00,-10.810698,47.658707,-32.282322,0.000000,0.000000,0.000000,0,0,0.000000,2,
3,0,5,0,1,3,0,0,0.000000,0.000003,0.000003,10,11,0.617299,0.604746,-0.503214,0.000002,1.922243e-06,-1.599515e-06,105.939369,36.316917,-103.856384,-0.534959,0.011966,-1.059854,999,1,12.832900,5,Scintillation
4,0,5,0,1,4,0,0,0.000000,0.000003,0.000003,10,-1,0.542201,-0.403473,-0.737040,0.000001,-1.056186e-06,-1.929379e-06,-0.512922,-0.004432,-1.089809,-0.534959,0.011966,-1.059854,999,1,466.209144,8,Scintillation
5,0,5,0,1,5,0,0,0.000000,0.000003,0.000003,10,-1,0.983522,0.034949,-0.177377,0.000003,9.238659e-08,-4.688887e-07,0.685928,0.055350,-1.280039,-0.534959,0.011966,-1.059854,999,1,136.674311,12,Scintillation


In [10]:
hittimes_df[(hittimes_df['EvtNum'].values == 4) & 
                     (hittimes_df['PhotonCreatorProcessName'] != "darkNoise")].sort_values('PrimaryParentID').head()

,Run,Date,EvtNum,SubEvtNumber,chitt,Truetime,PrimaryParentID,PhotonStartTime,PhotonStartPos_x0,PhotonStartPos_x1,PhotonStartPos_x2,PhotonEndPos_x0,PhotonEndPos_x1,PhotonEndPos_x2,PhotonCreatorProcessName
1230,0,10,4,1,99,66.980842,7,60.070583,-0.523495,-2.665611,-2.863837,-527.485291,192.443085,-1387.911743,Scintillation
1661,0,10,4,1,530,987.857574,7,979.650818,-0.527307,-2.665082,-2.864164,-1128.180542,-901.169434,983.405823,Scintillation
1530,0,10,4,1,399,179.763562,7,162.132874,-0.509258,-2.662219,-2.856508,-100.305618,1336.076782,1301.167725,Scintillation
1502,0,10,4,1,371,345.649484,7,339.145020,-0.514960,-2.663577,-2.859443,498.964813,1297.181152,28.500868,Scintillation
1285,0,10,4,1,154,96.573123,7,88.420601,-0.522259,-2.665316,-2.863200,-420.493042,-870.368713,-1464.284668,Scintillation


In [64]:
# Functions for the filtering
def electrons_from_Scintillation(df, nevents):

    # Take into account not only electrons may produce Scintillation, but for simplicity reasons we leave this like that.

    # Create a dictionary where the key is the event number and the info are the electrons that produced 
    # Scintillation light that produced hits in our PMTs.

    # Initialize the dictionary that's gonna record the info
    electrons_that_produce_scintillation = {}

    # Loop over all events
    for i in nevents:
        temp_df = df[df['EvtNum'].values == i]
        
        # If the event only has one track, append the track parent in a special way for the code not to break
        if len(temp_df) == 1:
            electrons_that_produce_scintillation[i] = list(temp_df['PrimaryParentID'].values)

        # In every other scenario, append the track parents unique appearances
        else:
            electrons_that_produce_scintillation[i] = list(np.unique(temp_df['PrimaryParentID'].values))

    return electrons_that_produce_scintillation

def real_scintillation_electrons(df, e, nevents):

    # Create a dictionary where the key is the event number and the info are the electron which produced
    # Scintillation light (that produced hits) that comes from the tag gamma

    # Initialize the dictionary
    el = {}

    # Loop over the events in which we have scintillation
    for i in tqdm(nevents, desc="Running real_scintillation_electrons", unit="iter", dynamic_ncols=True):

        # Create a sub-DataFrame with the data of the current event.
        # This speeds up the process 
        temp_df = df[df['EvtNum'].values == i]

        # Fill the dictionary with the track ID of those electrons that come from track 2, 
        # which is the tag gamma. Again, creating a temp DF is faster.
        el_temp = temp_df[temp_df['Id'].isin(e[i])]
        el[i]   = sorted(el_temp[el_temp['ParentId'].values == 2]['Id'].values)

    # Finally, if the dictionary is empty for one of these events, drop that index
    re = {}
    for i in nevents:
        if len(el[i]) == 0:
            pass
        else:
            re[i] = el[i]

    return re

def scintillation_info(e, df):
    events_scint  = []
    counts_scint  = []
    indices_scint = []

    # Loop over the events in which we have electrons from tag gamma
    for i in tqdm(e.keys(), desc="Running scintillation_info", unit="iter", dynamic_ncols=True):
        count = 0

        # Create a sub-DataFrame with the info of this event
        temp_df = df[df['EvtNum'] == i]
        indices = temp_df['PhotonIDParent'].index
        values  = temp_df['PhotonIDParent']

        # Loop over the hit parent of the digihits in the event and the index in the DF
        for j,l in zip(values, indices):
            # Loop over the electrons that come from the tag gamma in this event
            for k in e[i]:
                # If we have DigiHits for the current event and the electron appears in the
                # DigiHits DataFrame, count and append the index
                if temp_df.notna().any()[1] and k in j:
                    count += 1
                    indices_scint.append(l)

        # If we are in a event in which some of the DigiHits are created by the tag gamma,
        # append the events and the counts
        if count != 0:
            events_scint.append(i)
            counts_scint.append(count)

    return events_scint, counts_scint, indices_scint

def plot_light(data, bins, gamma_type, light_type, events, threshold, sfm, path, plot=False, save=True):
        fig = plt.figure(figsize=(8,6))

        plt.hist(data, bins=bins, color='green', alpha=0.4)
        plt.xlabel('# of DigiHits produced by {} Gamma {} Hits'.format(gamma_type, light_type))
        plt.yscale('log')
        plt.title('TriggerNDigits/Threshold == {} \n /DAQ/SaveFailures/Mode {}'.format(threshold, sfm))

        x_limits = plt.gca().get_xlim()
        y_limits = plt.gca().get_ylim()

        plt.annotate("In {} events we have DigiHits produced by".format(len(events)), xy=(0.10, 0.95), xycoords='axes fraction')
        plt.annotate("the {} light from the {} Gamma".format(light_type, gamma_type), xy=(0.10, 0.90), xycoords='axes fraction')
        plt.annotate("Average number of DigiHits per event is {:.0f}".format(np.mean(data)), xy=(0.10, 0.80), xycoords='axes fraction')
        plt.annotate("Max number of DigiHits per event is {:.0f}".format(np.max(data)), xy=(0.10, 0.75), xycoords='axes fraction')

        if plot:
            plt.show()

        if save:
            plt.savefig(path)

        plt.close

        return 0
    
def electrons_from_Cherenkov(df, nevents):

    # Create a dictionary where the key is the event number and the info are the electrons that produced 
    # Cherenkov light that produced hits in our PMTs.

    # Initialize the dictionary that's gonna record the info
    electrons_that_produce_cherenkov = {}

    # Loop over all events
    for i in nevents:
        temp_df = df[df['EvtNum'].values == i]
        
        # If the event only has one track, append the track parent in a special way for the code not to break
        if len(temp_df) == 1:
            electrons_that_produce_cherenkov[i] = list(temp_df['PrimaryParentID'].values)

        # In every other scenario, append the track parents unique appearances
        else:
            electrons_that_produce_cherenkov[i] = list(np.unique(temp_df['PrimaryParentID'].values))

    return electrons_that_produce_cherenkov

def real_cherenkov_electrons(df, e, nevents):

    # Create a dictionary where the key is the event number and the info are the electron which produced
    # Cherenkov light (that produced hits) that comes from the nCapture

    # Initialize the dictionary
    el = {}

    # Loop over the events in which we have nCapture and Cherenkov
    for i in tqdm(nevents, desc="Running real_cherenkov_electrons", unit="iter", dynamic_ncols=True):
        # Create a sub-DataFrame with the data of the current event.
        # This incredibly speeds up the process 
        temp_df = df[df['EvtNum'].values == i]

        # Look for the track ID of the nCapture Gamma in this event
        temp_ncID_df = temp_df[(temp_df['Ipnu'].values == 22)]
        ncID         = temp_ncID_df[(temp_ncID_df['CreatorProcessName'].values == "nCapture")]['Id'].values[0]

        # Take only the electrons which have been produced by the nCapture gamma
        temp_df = temp_df[(temp_df['Id'].isin(e[i]))]
        el[i] = sorted(temp_df[(temp_df['ParentId'].values == ncID)]['Id'].values)

    re = {}
    for i in nevents:
        if len(el[i]) == 0:
            pass
        else:
            re[i] = el[i]

    return re

def anyCherenkov_info(e, df):
    counts_anyCher = []
    events_anyCher = []

    for i in tqdm(e.keys(), desc="Running anyCherenkov info", unit="iter", dynamic_ncols=True):
        count = 0

        temp_df = df[df['EvtNum'].values == i]

        for j in temp_df['PhotonIDParent']:
            for k in e[i]:
                if temp_df.notna().any()[1] and k in j:
                    count += 1

        if count != 0:
            events_anyCher.append(i)
            counts_anyCher.append(count)

    return events_anyCher, counts_anyCher

def nCapture_Cherenkov_info(e, df):
    events_nCCher  = []
    counts_nCCher  = []
    indices_nCCher = []

    # Loop over the events in which we have electrons from the nCapture gamma
    for i in tqdm(e.keys(), desc="Running nCapture_Cherenkov_info", unit="iter", dynamic_ncols=True):
        count = 0
        # Create a sub-DataFrame of the current event
        temp_df = df[df['EvtNum'].values == i]

        # Loop over the hit parents that created the DigiHits and the index
        for j, l in zip(temp_df['PhotonIDParent'], temp_df['PhotonIDParent'].index):
            # Loop over the electrons created by the nCapture gamma in this event
            for k in e[i]:
                # If the electron is in the DigiHits DF, append the index and count
                if temp_df.notna().any()[1] and k in j:
                    indices_nCCher.append(l)
                    count += 1

        # If we are in an event we want to append, do it and print info           
        if count != 0:
            events_nCCher.append(i)
            counts_nCCher.append(count)

    return events_nCCher, counts_nCCher, indices_nCCher

# Scintillation Light Filtering

In [ ]:
events_with_Scintillation = np.unique(tracks_df[tracks_df['CreatorProcessName'].values == "Scintillation"]['EvtNum'])
sHits_Tag_temp            = hittimes_df[(hittimes_df['EvtNum'].isin(events_with_Scintillation))]
sHits_Tag                 = hittimes_df[(hittimes_df['PhotonCreatorProcessName'].values == "Scintillation")]

scint_nevents = list(sHits_Tag.groupby("EvtNum").count().index.values)

In [70]:
electrons_that_produce_scintillation      = test_filters.electrons_from_Scintillation(sHits_Tag, scint_nevents)
electrons_from_tagGamma                   = test_filters.real_scintillation_electrons(tracks_df, electrons_that_produce_scintillation, scint_nevents)
events_scint, counts_scint, indices_scint = test_filters.scintillation_info(electrons_from_tagGamma, digihits_df)

Running real_scintillation_electrons:   0%|                                                                   …

Running scintillation_info:   0%|                                                                             …

# Cherenkov Light Filtering

In [71]:
events_with_nCapture = np.unique(tracks_df[tracks_df['CreatorProcessName'] == 'nCapture']['EvtNum'].to_numpy())
temp_cHits_nCapture  = hittimes_df[hittimes_df['EvtNum'].isin(events_with_nCapture)]
cHits_nCapture       = temp_cHits_nCapture[temp_cHits_nCapture['PhotonCreatorProcessName'].values == 'Cerenkov']

cher_nevents = cHits_nCapture.groupby('EvtNum').count().index.values

In [72]:
electrons_that_produce_cherenkov             = test_filters.electrons_from_Cherenkov(cHits_nCapture, cher_nevents)
electrons_from_nCapture                      = test_filters.real_cherenkov_electrons(tracks_df, electrons_that_produce_cherenkov, cher_nevents)
# events_anyCher, counts_anyCher               = test_filters.anyCherenkov_info(electrons_that_produce_cherenkov, digihits_df)
events_nCCher, counts_nCCher, indices_nCCher = test_filters.nCapture_Cherenkov_info(electrons_from_nCapture, digihits_df)

Running real_cherenkov_electrons:   0%|                                                                       …

Running nCapture_Cherenkov_info:   0%|                                                                        …

# Create Final DigiHits DataFrame

In [73]:
# Creating a new DigiHits DataFrame just with the Tag and nCapture light
df_cher  = digihits_df.loc[indices_nCCher]
df_scint = digihits_df.loc[indices_scint]

cher_events  = np.unique(df_cher['EvtNum'])
scint_events = np.unique(df_scint['EvtNum'])

common_evts  = [i for i in np.unique(digihits_df['EvtNum']) if i in cher_events and i in scint_events]
print("There are {} events with both Tag and nCapture light".format(len(common_evts)))
print(" ")

cher_indices  = df_cher[df_cher['EvtNum'].isin(common_evts)].index.to_numpy()
scint_indices = df_scint[df_scint['EvtNum'].isin(common_evts)].index.to_numpy()

final_indices = np.sort(np.concatenate([cher_indices, scint_indices]))

df_final       = digihits_df.loc[final_indices]
df_final_cher  = df_cher[df_cher['EvtNum'].isin(common_evts)]
df_final_scint = df_scint[df_scint['EvtNum'].isin(common_evts)]

There are 340 events with both Tag and nCapture light
 


# SI NOS QUEDAMOS AQUÍ, TENEMOS TODOS LOS HITS DE CHERENKOV DE LA nCAPTURE. SI SEGUIMOS ADELANTE, NOS QUEDAMOS SOLO CON AQUELLOS EN UNA VENTANA DE 250 us DESPUÉS DE QUE EL ÚLTIMO HIT DE TAG SEA RECOGIDO.

# IGUAL PARA ESTA RECONSTRUCCIÓN BASTA CON LLEGAR A ESTE PUNTO? YA QUE LA VENTANA Y TODO ESO ES PARA EL ALGORITMO DE NEUTRON CANDIDATE.

In [74]:
# last_tag_hit_time = df_final_scint.groupby('EvtNum')['T'].max()
# repeated_last_tag_hit_time = df_final_cher['EvtNum'].map(last_tag_hit_time)

# # Agregamos la columna 'last_tag_hit_time' a df_final_cher
# df_final_cher.loc[:, 'last_tag_hit_time'] = repeated_last_tag_hit_time

# # Nuevo DataFrame con la luz de cherenkov solo en la ventana
# filter_df_final_cher = df_final_cher.loc[(df_final_cher['T'] >= df_final_cher['last_tag_hit_time']) &
#                                          (df_final_cher['T'] < df_final_cher['last_tag_hit_time']+250000)]

# # Creamos el DataFrame de DarkNoise
# indices = []

# for i in tqdm(np.unique(filter_df_final_cher['EvtNum'])):
#     temp_df = digihits_df[digihits_df['EvtNum'] == i]
#     tag_t   = filter_df_final_cher[filter_df_final_cher['EvtNum'].values == i]['last_tag_hit_time'].values[0]
    
#     for p, ind, cher_t in zip(temp_df['PhotonIDParent'], temp_df['PhotonIDParent'].index, temp_df['T']):
#         if -1 in p and cher_t >= tag_t and cher_t < tag_t + 250000:
#             indices.append(ind)
            
# bkg_df = digihits_df.loc[indices]

# # Y finalmente el DataFrame último con solo luz de cherenkov
# events_with_no_dr           = [i for i in np.unique(filter_df_final_cher['EvtNum']) if i not in np.unique(bkg_df['EvtNum'])]
# filter_df_final_cher_events = np.unique(filter_df_final_cher['EvtNum'])

# final_events = [i for i in filter_df_final_cher_events if i not in events_with_no_dr]
# final_data   = filter_df_final_cher[filter_df_final_cher['EvtNum'].isin(final_events)]

# merged_df = pd.concat([filter_df_final_cher, bkg_df])
# merged_df.sort_values(by='EvtNum', inplace=True)

# WRITING INFO BACK
Take into account that we are going with df_final_cher instead of df_cher. The difference between both is that the former only has the the Cherenkov light in the events in which we can see tag and ncapture light, i.e., the ones that we would be able to tag in the lab. This is nice but also makes the cherenkov light sample smaller.

In [75]:
df_final_cher = df_final_cher.drop(["PhotonIDParent"], axis=1)

In [76]:
df_final_cher.head()

,Run,Date,EvtNum,SubEvtNumber,dhit,Q,T,TubeID,mPMTID,mPMT_PMTID
781,0,92890,4,10,0,3.015210,93842.5,1272,67,18
782,0,92890,4,10,1,0.914677,93841.0,552,30,1
783,0,92890,4,10,2,1.241733,93841.7,1183,63,5
784,0,92890,4,10,3,0.413416,93839.7,1182,63,4
785,0,92890,4,10,4,1.435695,93840.7,999,53,11


In [77]:
# WRITING USING PYTABLES
# import pandas as pd
# import tables as tb

# # Definir la ruta del archivo HDF5 de salida
# output_file = "/Users/diiego/software/WCSimDataTools/filter_digihits_df.h5"

# # Crear el archivo HDF5 y los grupos necesarios
# with tb.open_file(output_file, mode='w', title="wcsim") as hdf_file:
#     # Settings
#     table_settings = hdf_file.create_table("/", "Settings", description=settings_df.dtype, title="Settings")
#     table_settings.append(settings_df)
    
#     # wcsimRootOptionsT
#     table_roptions = hdf_file.create_table("/", "wcsimRootOptionsT", descriptions.Options, "wcsimRootOptionsT")
#     table_roptions.append([tuple(roptions_df.values[0])])
    
#     # wcsimGeoT
#     group_wcsimGeoT = hdf_file.create_group("/", "wcsimGeoT", "wcsimGeoT")
    
#     # wcsimGeoT/Geometry
#     table_geometry = hdf_file.create_table(group_wcsimGeoT, "Geometry", description=geometry_df.dtype, title="Geometry")
#     table_geometry.append(geometry_df)
    
#     # wcsimGeoT/PMT
#     table_pmt = hdf_file.create_table(group_wcsimGeoT, "PMT", descriptions.PMT, "PMT")
#     table_pmt.append(pmt_df.to_records(index=False))
    
#     # wcsimT
#     group_wcsimT = hdf_file.create_group("/", "wcsimT", "wcsimT")
    
#     # /wcsimT/CherenkovHitTimes
#     table_hittimes = hdf_file.create_table(group_wcsimT, "CherenkovHitTimes", descriptions.CherenkovHitTimes, "CherenkovHitTimes")
#     table_hittimes.append(filter_hittimes_df.to_records(index=False))
    
#     # /wcsimT/CherenkovHits
#     table_cherhits = hdf_file.create_table(group_wcsimT, "CherenkovHits", descriptions.CherenkovHits, "CherenkovHits")
#     table_cherhits.append(filter_cherhits_df.to_records(index=False))
    
#     # wcsimT/Tracks
#     table_tracks = hdf_file.create_table(group_wcsimT, "Tracks", descriptions.Tracks, "Tracks")
#     table_tracks.append(filter_tracks_df.to_records(index=False))
    
#     # wcsimT/CherenkovDigiHits
#     group_CherenkovDigiHits = hdf_file.create_group(group_wcsimT, "CherenkovDigiHits", "CherenkovDigiHits")

#     # wcsimT/CherenkovDigiHits/DigiHits
#     table_digihits = hdf_file.create_table(group_CherenkovDigiHits, "DigiHits", descriptions.CherenkovDigiHits, "DigiHits")
#     table_digihits.append(filter_digihits_df.to_records(index=False))
    
#     # wcsimT/CherenkovDigiHits/PhotonIDs
#     table_photonid = hdf_file.create_vlarray(group_CherenkovDigiHits, "PhotonIDs", a, title="PhotonIDs")
#     table_photonid.append(a)
    
#     # wcsimT/Triggers
#     table_triggers = hdf_file.create_table(group_wcsimT, "Triggers", descriptions.Triggers, "Triggers")
    
#     # Make the strings a dtype available for HDF5
#     arrays    = [triggers_df[col].to_numpy() for col in triggers_df.columns]
#     formats   = [array.dtype if array.dtype != 'O' else array.astype("S50").dtype for array in arrays] 
#     triggers_rec_array = np.rec.fromarrays(arrays, dtype={'names': triggers_df.columns, 'formats': formats})
    
#     table_triggers.append(triggers_rec_array)
    
#     # Cerrar el archivo
#     hdf_file.close()

# ACTUAL WRITER FUNCTION BEING CALLED FROM .py

In [78]:
writer.write("/Users/diiego/software/WCSimDataTools/filtered_ambeSourceData.hdf5", settings_df, roptions_df, geometry_df, pmt_df, hittimes_df, cherhits_df,
            tracks_df, triggers_df, df_final_cher, photonid_df, photparn_df)

In [79]:
!h5ls -rl /Users/diiego/software/WCSimDataTools/filtered_ambeSourceData.hdf5

/                        Group
/Settings                Dataset {1}
/wcsimGeoT               Group
/wcsimGeoT/Geometry      Dataset {1}
/wcsimGeoT/PMT           Dataset {2014, 10}
/wcsimRootOptionsT       Dataset {1}
/wcsimT                  Group
/wcsimT/CherenkovDigiHits Group
/wcsimT/CherenkovDigiHits/DigiHits Dataset {3059, 10}
/wcsimT/CherenkovDigiHits/DigiHitsPhotonIdParent Dataset {158470}
/wcsimT/CherenkovDigiHits/PhotonIDs Dataset {158470}
/wcsimT/CherenkovHitTimes Dataset {362937}
/wcsimT/CherenkovHits    Dataset {320753, 10}
/wcsimT/Tracks           Dataset {4481925}
/wcsimT/Triggers         Dataset {10780}
